In [1]:
import requests
import json
import matplotlib.pyplot as plt
import datetime
import pandas as pd
from datetime import datetime
from datetime import timedelta
from fabric import Connection
import psycopg2
import gmaps
import pandas.io.sql as sqlio
import getpass
from io import StringIO
import paramiko
import sys
from time import sleep

sys.path.insert(0, "/home/jovyan/watexr/PROGNOS/")
from prognos_tools.gce_light import gce_api as gce

key = getpass.getpass('mobiserver password: ')
cloudKey = getpass.getpass('vault password: ')
#Querying necessary tokens
def query(query,fetch=True):
    with psycopg2.connect(user='jose-luis', host='mobiserver.niva.no', port=5432, database='vault',password=key) as db:
        with db.cursor() as cursor :
            cursor.execute(query)
            if fetch:
                result = sqlio.read_sql_query(query, db)
                return result
            
# def insertFile(alias,filename):
#     with psycopg2.connect(user='jose-luis', host='mobiserver.niva.no', port=5432, database='vault',password=key) as db:
#         with db.cursor() as cursor :
#             cursor.execute('drop table if exists dummy;')
#             cursor.execute('''create temp table dummy (sometext bytea);''')
#             with open(filename,'rb') as f:
# #                 cursor.copy_from(f,'dummy')
#                 cursor.execute('''insert into dummy  select %s::bytea;''',(psycopg2.Binary(f.read()),) )
#             cursor.execute('''select niva.insertToken('{}',(select * from dummy),'');'''.format(alias));

gmapsKey = query('''select niva.getToken('gmaps','{}');'''.format(cloudKey)).iloc[0,0]
sshKey = query('''select niva.getToken('geonorway_ssh_key','{}');'''.format(cloudKey)).iloc[0,0]
cloudKey = json.loads(query('''select niva.getToken('gce_access','{}');'''.format(cloudKey)).iloc[0,0])
gmaps.configure(gmapsKey)
not_really_a_file = StringIO(sshKey)
private_key = paramiko.RSAKey.from_private_key(not_really_a_file)
del key,sshKey

mobiserver password:  ·······
vault password:  ···············


In [2]:
#Check status of instance
properties = {'project'      : 'nivacatchment',
             'zone'         : 'europe-north1-a',
             'instanceType' : "n1-standard-4",
             'instanceName' : "dtm10",
             'username'     : "jose-luis",
             }

cloud = gce(properties, cloudKey)
del cloudKey

In [3]:
#Getting instance info
cloud.CommonCalls['custom'] = '''https://compute.googleapis.com/compute/v1/projects/{project}/zones/{zone}/instances/{instanceName}'''
info = cloud.get('custom')
display(info['status'])
#If instance is stopped, start it
if info['status'] != 'RUNNING':
    cloud.CommonCalls['custom'] = '''https://compute.googleapis.com/compute/v1/projects/{project}/zones/{zone}/instances/{instanceName}/start'''
    info = cloud.post('custom')
    display(info['status'])
    cloud.CommonCalls['custom'] = '''https://compute.googleapis.com/compute/v1/projects/{project}/zones/{zone}/instances/{instanceName}'''
    info = cloud.get('custom')
    while info['status'] != 'RUNNING':
           sleep(2)
           info = cloud.get('custom')
          
geonorway = info['networkInterfaces'][0]['accessConfigs'][0]['natIP']    
    
display(geonorway)
config =  {'host' : geonorway, 'user': 'jose-luis', 'connect_kwargs': {'pkey': private_key } }

'RUNNING'

'35.228.213.48'

# Getting nve data

In [4]:
#Loading key
with open('/home/jovyan/malselva/nve_key','r') as f:
    key = f.read().strip()
    
#Listing all parameters measured by NVE
headers={'accept': 'application/json',
         'X-API-Key': '{}'.format(key)}

r = requests.get('https://hydapi.nve.no/api/v1/Parameters', headers=headers)

display(json.loads(r.text))                  

{'currentLink': 'https://hydapi.nve.no/api/v1/Parameters',
 'apiVersion': '1.0',
 'license': 'https://data.norge.no/nlod/en',
 'createdAt': '2021-03-15T10:36:36.5573187Z',
 'queryTime': '00:00:00.0000076',
 'itemCount': 31,
 'data': [{'parameter': 0,
   'parameterName': 'Nedbør',
   'parameterNameEng': 'Precipitation',
   'unit': 'mm'},
  {'parameter': 1000,
   'parameterName': 'Vannstand',
   'parameterNameEng': 'Stage',
   'unit': 'm'},
  {'parameter': 1001,
   'parameterName': 'Vannføring',
   'parameterNameEng': 'Discharge',
   'unit': 'm³/s'},
  {'parameter': 1002,
   'parameterName': 'Vannhastighet',
   'parameterNameEng': 'Water speed',
   'unit': 'm/s'},
  {'parameter': 1003,
   'parameterName': 'Vanntemperatur',
   'parameterNameEng': 'Water temperature',
   'unit': '°C'},
  {'parameter': 1004,
   'parameterName': 'Magasinvolum',
   'parameterNameEng': 'Reservoir volume',
   'unit': 'millioner m³'},
  {'parameter': 1006,
   'parameterName': 'Ledningsevne',
   'parameterNameEng

In [5]:
parameter = 1001
r = requests.get('https://hydapi.nve.no/api/v1/Stations', headers=headers)
data = json.loads(r.text)
stations_with_q = [{'stationName' : i['stationName'],
                   'stationId'    : i['stationId'],
                   'latitude'     : i['latitude'],
                   'longitude'    : i['longitude'],
                   'parameters'   : i['seriesList']
                   }
                  for i in data['data'] if parameter in [j['parameter'] for j in i['seriesList']]]

allStations = [{'stationName' : i['stationName'],
                   'stationId'    : i['stationId'],
                   'latitude'     : i['latitude'],
                   'longitude'    : i['longitude'],
                   'parameters'   : i['seriesList']
                   }
                  for i in data['data'] ]


In [6]:
with open('dummy.txt','w') as f:
    for ii in stations_with_q : 
        f.write(json.dumps(ii))
        f.write('\n\n')

In [7]:
len(stations_with_q)#data

603

In [8]:
schema = 'nveq'
def query(query,fetch=True):
    with psycopg2.connect(user='jose-luis', host=geonorway, port=5432, database='geonorway') as db:
        cursor = db.cursor()
        cursor.execute(query)
        if fetch:
            result = sqlio.read_sql_query(query, db)
            return result
        
query('''drop schema if exists {0} cascade;
create schema {0};'''.format(schema),fetch=False)

        
sql = '''create table {0}.stations (sid int generated always as identity,
stationName varchar(100) not null,
stationID varchar(50) unique not null,
latitude double precision not null,
longitude double precision not null,
unique(sid)
);

insert into {0}.stations (stationName,stationId,latitude,longitude) values {1};

create index {0}_stations_station_id_idx on {0}.stations(stationId);
alter table {0}.stations add column geom geometry(point,4326);
update {0}.stations
set geom = st_setsrid(st_makepoint(longitude,latitude),4326);
'''.format(schema,',\n'.join(["('" + i['stationName'].replace("'","''") + "',"
                     + "'" + i['stationId'] + "'" + ','
                     + str(i['latitude']) + ','
                     + str(i['longitude']) 
                     + ')'
                    for i in data['data'] if parameter in [j['parameter'] for j in i['seriesList']]
                    ]
                    )
          )

# with open('dummy.sql','w') as f:
#     f.write(sql)
query(sql,fetch=False)

In [9]:
#Storing metadata for all stations
sql = '''drop table if exists {0}.metadata;
create table {0}.metadata (mid int generated always as identity, sid int,  metadata json,
primary key(mid),
constraint nve_stations_k
foreign  key(sid)
references {0}.stations(sid)
);
'''.format(schema)
query(sql,fetch=False)

sids =  query('''select sid,stationId from {}.stations;'''.format(schema))

for ii,jj in sids.iterrows():
#     display(jj['sid'],jj['stationid'])
    r = requests.get('https://hydapi.nve.no/api/v1/Series?StationId={}'.format(jj['stationid']),headers=headers);
    display('{} -> {}'.format(ii,r.status_code))
    sleep(0.1)
    query('''insert into {}.metadata(sid,metadata) values ({},'{}')'''.format(schema,jj['sid'],r.text.replace("'","''")),fetch=False)
# len(sids)

'0 -> 200'

'1 -> 200'

'2 -> 200'

'3 -> 200'

'4 -> 200'

...

'598 -> 200'

'599 -> 200'

'600 -> 200'

'601 -> 200'

'602 -> 200'

In [10]:
# display(query('''select json_array_elements(metadata #> '{data}') from nvedata.metadata limit 10;'''))
# a = query('''select json_array_elements(metadata -> 'data')->'stationId' from nvedata.metadata limit 10;''')
query('''drop table if exists {}.singledata;'''.format(schema),fetch=False)
query('''create table {0}.singledata as select sid, json_array_elements(metadata -> 'data') as jdoc from {0}.metadata;'''.format(schema),fetch=False)
query('''alter table {}.singledata add column id int generated always as identity;'''.format(schema),fetch=False)
query('''alter table {}.singledata add primary key(id);'''.format(schema),fetch=False)
query('''alter table {0}.singledata add constraint nve_stations_k foreign key(sid) references {0}.stations(sid);'''.format(schema) ,fetch=False)
query('''alter table {}.singledata alter column jdoc set data type jsonb using jdoc::jsonb;'''.format(schema),fetch=False)
a = query('''select * from {}.singledata;'''.format(schema))
display(a)

,sid,jdoc,id
0,1,"{'masl': 177, 'unit': 'm', 'serieTo': None, 'l...",1
1,1,"{'masl': 177, 'unit': 'm³/s', 'serieTo': '2005...",2
2,1,"{'masl': 177, 'unit': '°C', 'serieTo': '2005-0...",3
3,1,"{'masl': 177, 'unit': '°C', 'serieTo': None, '...",4
4,2,"{'masl': 134, 'unit': 'm', 'serieTo': None, 'l...",5
...,...,...,...
1828,602,"{'masl': 462, 'unit': 'm³/s', 'serieTo': None,...",1829
1829,602,"{'masl': 462, 'unit': '°C', 'serieTo': None, '...",1830
1830,603,"{'masl': 563, 'unit': 'm', 'serieTo': None, 'l...",1831
1831,603,"{'masl': 563, 'unit': 'm³/s', 'serieTo': None,...",1832


In [11]:
query('''drop table if exists {0}.dummy'''.format(schema),fetch=False)
query('''create table {0}.dummy as select distinct a.sid,
(a.jdoc->>'latitude')::float as latitude, 
(a.jdoc->>'longitude')::float as longitude
,b.stationName, b.stationId, jsonb_array_elements(a.jdoc -> 'resolutionList') -> 'resTime' as resTime, 
jsonb_array_elements(a.jdoc -> 'resolutionList') -> 'dataFromTime' as fromTime,
jsonb_array_elements(a.jdoc -> 'resolutionList') -> 'dataToTime' as toTime,
a.jdoc -> 'parameter' as parameter
from {0}.singledata as a join {0}.stations as b on b.sid=a.sid 
where jdoc @>'{{"parameter" : {1} }}'
'''.format(schema,parameter),fetch=False)
water_temp = query('''select * from {}.dummy where resTime='0' '''.format(schema))
display(water_temp)

,sid,latitude,longitude,stationname,stationid,restime,fromtime,totime,parameter
0,1,59.92993,11.40768,Berg,1.198.0,0,1991-06-24T09:00:00Z,2005-07-22T12:00:00Z,1001
1,2,59.92038,11.53180,Lierelv,1.200.0,0,2010-09-15T15:00:00Z,2021-03-15T05:00:00Z,1001
2,3,59.48267,11.65056,Ørje,1.48.0,0,1963-09-01T11:00:00Z,2021-03-07T11:00:00Z,1001
3,4,59.14763,11.55638,Brekke,1.49.0,0,2020-12-13T11:00:00Z,2021-03-07T11:00:00Z,1001
4,5,59.12735,11.44301,Tistedalsfoss,1.50.0,0,2020-12-13T11:00:00Z,2021-03-07T11:00:00Z,1001
...,...,...,...,...,...,...,...,...,...
603,599,62.07259,6.45240,Sleddalen,97.5.0,0,1997-12-16T14:00:00Z,2021-03-15T10:00:00Z,1001
604,600,62.06818,6.93163,Øye ndf.,98.4.0,0,1990-10-02T10:30:00Z,2021-03-15T10:00:00Z,1001
605,601,62.21919,7.42720,Onilsavatn,99.1.0,0,1943-10-20T11:00:00Z,1975-08-18T11:00:00Z,1001
606,602,62.19769,7.49543,Rødøla,99.17.0,0,1981-05-01T11:00:00Z,2021-01-31T23:00:00Z,1001


In [12]:
query('''alter table {0}.dummy add column geom geography(point,4326);
update {0}.dummy set geom = ST_SetSRID(ST_MakePoint(longitude, latitude),4326);'''.format(schema),fetch=False)

In [13]:
a = query('''select a.stationname,a.stationid,a.fromtime,a.totime, st_distance( a.geom, ST_SetSRID(ST_MakePoint(7.5281230, 59.0976410),4326) ) / 1000 as distance from 
{}.dummy as a where restime = '0' order by st_distance(a.geom,ST_SetSRID(ST_MakePoint(7.5281230, 59.0976410),4326) );'''.format(schema))
a.to_csv('nve_temp.csv')

## Displaying stations with discharge

In [14]:
fig = gmaps.figure(map_type="TERRAIN")
info_box_template = """
    <dl>
    <font color="black">
    <dt>Name</dt><dd>{stationname}</dd>
    <dt>Id</dt><dd>{stationid}</sup></dd>
    <dt>fromTime</dt><dd>{fromtime}</sup></dd>
    <dt>toTime</dt><dd>{totime}</sup></dd>
    <dt>Long</dt><dd>{longitude:.8f}</dd>
    <dt>Lat</dt><dd>{latitude:.8f}</dd>
    </font>
    </dl>
    """  
locations = [(i,j) for i,j in zip(water_temp['latitude'],water_temp['longitude'])]
station_info = [info_box_template.format(**i[1]) for i in water_temp.iterrows()]                                                 
marker_layer = gmaps.marker_layer(locations, label=water_temp['stationname'].values, info_box_content=station_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
#Redefining query()
def query(query,fetch=True):
    with psycopg2.connect(user='jose-luis', host=geonorway, port=5432, database='geonorway') as db:
        cursor = db.cursor()
        cursor.execute(query)
        if fetch:
            result = sqlio.read_sql_query(query, db)
            return result

sids = list(water_temp['stationid'].values)
# r = requests.get('https://hydapi.nve.no/api/v1/Series?StationId={}'.format(sid),headers=headers);
# display(r)
# info = json.loads(r.text)
# display(info)

# schema = 'nvedata'
# query('drop schema if exists {} cascade;'.format(schema),fetch=False)
# query('create schema {};'.format(schema),fetch=False)

# #Creating tables with nve ids
# query('''drop table if exists {}.nveStations cascade;'''.format(schema),fetch=False)
# query('''create table {}.nveStations(id serial unique, nveId varchar(50),primary key(id));'''.format(schema),fetch = False)
# query('''insert into {}.nveStations (nveId) values {} '''.format(schema,','.join(["('" + i + "')" for i in sids])),fetch=False)

In [ ]:
table_name = 'discharge'

sql = '''drop table if exists {0}.{1};
create table {0}.{1}(sid int,fecha timestamptz,{1} double precision,correction double precision, quality integer, unique(fecha,{1},correction,quality),
constraint idk
foreign key(sid)
references {0}.stations(sid));
'''.format(schema,table_name)

with open('q.sql','w') as f:
    f.write(sql)
with Connection(**config) as c:
    c.run('rm -f q.sql')
    c.put('q.sql')
    c.run('psql -q -d geonorway -f q.sql')
    c.run('rm -f q.sql')
    
def raw_to_string(i,currentId):
    return f'''({str(currentId)},
    '{datetime.strptime(i['time'],('%Y-%m-%dT%H:%M:%SZ')).strftime('%Y-%m-%d %H:%M:%S') + '+00' }',
    {i['value'] or 'Null'},
    {i['correction']},
    {i['quality']})'''


def date_range(start, end, intv):    
    start = datetime.strptime(start,"%Y-%m-%d")
    end = datetime.strptime(end,"%Y-%m-%d")
    diff = (end  - start ) / intv
    for i in range(intv):
        yield (start + diff * i)#.strftime("%Y-%m-%d")
    yield end#.strftime("%Y-%m-%d")
    

def splitter(length,base=1,power=2):
    while True:
        for i in range(length):
            yield base * power**i
    


for i,j in water_temp.iterrows():
    display('Downloading data for {}'.format(str(j['stationid'])));
    parameter = j['parameter']
    restime = j['restime']
    referencetime = '{}/{}'.format(j['fromtime'].split('T')[0],j['totime'].split('T')[0])
    currentId = j['stationid']
    
    #First attempt at downloading the data
    r = requests.get('https://hydapi.nve.no/api/v1/Observations',headers=headers,params={'StationId': currentId, 
                                                                                             'Parameter': parameter, 
                                                                                             'ResolutionTime' : j['restime'],
                                                                                             'ReferenceTime' : referencetime})
    
    if r.status_code == 200:
        raw_data = json.loads(r.text)['data'][0]['observations']
        if len(raw_data) > 0 :
            values = [raw_to_string(l,j['sid']) for l in raw_data if raw_to_string(l,j['sid']) is not None]
            with open('q.sql','w') as f:
                f.write('''insert into {}.{} values {} on conflict do nothing;'''.format(schema,table_name,','.join(values)))
                f.write('\n')
            with Connection(**config) as c:
                c.run('rm -f q.sql')
                c.put('q.sql')
                c.run('psql -q -d geonorway -f q.sql')
                c.run('rm -f q.sql')
    else :
        
        start_date = referencetime.split('/')[0] 
        num_chunks = splitter(12)
        next(num_chunks) #So we start at two and not at one
        while r.status_code != 200: #Keep on trying until the command succeeds. Probably not the best of ideas since this means all the edge cases need to be handled below
            if r.text.startswith('"The query gives more than 150000 observations'):  #Other conditions will potentially need to be added if some other errors are generated
                #Splitting the data
                #There might be too many data within the reference time so we split it in chunks if that happens
                #The chunks  become increasingly smaller if the "too much data" error persits   
                splits=list(date_range(start_date,referencetime.split('/')[1],next(num_chunks)))
                intervals = [[i,j] for i,j in zip(splits[:-1],splits[1:])]
                intervals[1:] = [[i[0] ,i[1]] for i in intervals[1:]]
            for k in intervals:
                timechunk = '{}/{}'.format(k[0].strftime('%Y-%m-%d'),k[1].strftime('%Y-%m-%d'))
                display("Downloading period : {}".format(timechunk))
                r = requests.get('https://hydapi.nve.no/api/v1/Observations',headers=headers,params={'StationId': currentId, 
                                                                                                     'Parameter': parameter, 
                                                                                                     'ResolutionTime' : j['restime'],
                                                                                                     'ReferenceTime' : timechunk})
                if r.status_code == 200:
                    raw_data = json.loads(r.text)['data'][0]['observations']
                    if len(raw_data) > 0 :
                        values = [raw_to_string(l,j['sid']) for l in raw_data if raw_to_string(l,j['sid']) is not None]
                        with open('q.sql','w') as f:
                            f.write('''insert into {}.{} values {} on conflict do nothing;'''.format(schema,table_name,','.join(values)))
                            f.write('\n')
                        with Connection(**config) as c:
                            c.run('rm -f q.sql')
                            c.put('q.sql')
                            c.run('psql -q -d geonorway -f q.sql')
                            c.run('rm -f q.sql')
                else : 
#                     if k[1]!=k[0]:
                    display("Failed for the current interval, resetting starting point and number of chunks")
                    start_date = (k[0] - timedelta(days=1)).strftime('%Y-%m-%d')
                    break
                
                
#             display(r.text)
#                 sleep(0.25)
                
                      
        



#     display(json.loads(r.status))

#     splits=list(date_range(referencetime.split('/')[0],referencetime.split('/')[1],50))
#     intervals = [[i,j] for i,j in zip(splits[:-1],splits[1:])]
#     intervals[1:] = [[i[0] + timedelta(days=1),i[1]] for i in intervals[1:]]

    
#     currentId = j['stationid']
    
# #     #Remember that nve time information is given in zulu time
#     for k in intervals:
#         timechunk = '{}/{}'.format(k[0].strftime('%Y-%m-%d'),k[1].strftime('%Y-%m-%d'))
# #         display("Downloading period : {}".format(timechunk))
#         r = requests.get('https://hydapi.nve.no/api/v1/Observations',headers=headers,params={'StationId': currentId, 
#                                                                                              'Parameter': parameter, 
#                                                                                              'ResolutionTime' : j['restime'],
#                                                                                              'ReferenceTime' : timechunk})
# #         display(r.text)
# #         sleep(0.25)
#         while (r.status_code != 200):
#             display(r.text)
#             sleep(2)
#             r = requests.get('https://hydapi.nve.no/api/v1/Observations',headers=headers,params={'StationId': currentId, 
#                                                                                              'Parameter': parameter, 
#                                                                                              'ResolutionTime' : j['restime'],
#                                                                                              'ReferenceTime' : timechunk})
        
#         raw_data = json.loads(r.text)['data'][0]['observations']
        
#         if len(raw_data) > 0 :
#             values = [raw_to_string(l,j['sid']) for l in raw_data if raw_to_string(l,j['sid']) is not None]
#             with open('q.sql','w') as f:
#                 f.write('''insert into {}.watertemp values {};'''.format(schema,','.join(values)))
#                 f.write('\n')
#             with Connection(**config) as c:
#                 c.run('rm -f q.sql')
#                 c.put('q.sql')
#                 c.run('psql -q -d geonorway -f q.sql')
#                 c.run('rm -f q.sql')

        
        
with open('q.sql','w') as f:
    f.write('drop index if exists {}_fecha_idx;'.format(schema))
    f.write('\n')
    f.write('create index {0}_fecha_idx on {0}.{1} using btree(fecha);'.format(schema,table_name))
    f.write('\n')
    f.write('create index {0}_id_idx on {0}.{1}(sid);'.format(schema,table_name))
with Connection(**config) as c:
    c.run('rm -f q.sql')
    c.put('q.sql')
    c.run('psql -q -d geonorway -f q.sql')
    c.run('rm -f q.sql')
    
        
display('Done')

'Downloading data for 1.198.0'

'Downloading data for 1.200.0'

'Downloading data for 1.48.0'

'Downloading data for 1.49.0'

'Downloading data for 1.50.0'

...

'Downloading period : 1995-01-12/2001-07-28'

'Downloading period : 2001-07-28/2008-02-12'

'Downloading period : 2008-02-12/2014-08-29'

'Downloading period : 2014-08-29/2021-03-15'

'Downloading data for 124.15.0'

In [30]:
# water_temp[5:]
k[0] -timedelta(days=1)

datetime.datetime(2002, 8, 8, 0, 0)

In [ ]:
#### Putting into database
with Connection(**config) as c:
    c.run('rm -f q.sql')
    c.put('q.sql')
    c.run('psql -q -d geonorway -f q.sql')
    c.run('rm -f q.sql')

In [ ]:
a = query('''select distinct a.sid,b.stationName from nvedata.watertemp as a join nvedata.stations as b on a.sid=b.sid order by a.sid;''')
columns = ' double precision,'.join([i.replace(' ','_').replace('/','_').replace("'",'_').replace('.','_') for i in a['stationname'].values]) + ' double precision'
a=query('''select * from crosstab
('select fecha, sid, t from nvedata.watertemp limit 20
order by 1,2',
'select distinct sid from nvedata.watertemp order by 1' 
) as f(fecha timestamptz, {})
;
'''.format(columns))
a.set_index('fecha',inplace=True)
a.plot(subplots=True,figsize=(20,15))
plt.xlim()

In [ ]:
a = query('''select sid,stationname,stationid from nvedata.stations where sid != 1;''')
for i,j in a.iterrows():
    data = query('''select fecha, t from nvedata.watertemp where sid={};'''.format(j['sid']))
    data.rename(columns={'fecha' : 'date', 't': '{}/{}'.format(j['stationname'],j['stationid'])},inplace=True)
    data.set_index('date').plot(figsize=(20,5))
    plt.savefig('figs/{}.pdf'.format(j['sid']))
    plt.close

In [40]:
a = query('''select count(*) from (select distinct * from nveq.discharge where sid=7) as bla;''')
a
# a.set_index('fecha').plot(figsize=(20,5))

,count
0,258683


In [ ]:
water_temp = query('''select * from nvedata.dummy where resTime='0' ''')
water_temp.set_index('sid').to_csv('stations_with_temp.csv')

In [ ]:
def splitter(length,base=1,power=2):
    while True:
        for i in range(length):
            yield base * power**i

In [ ]:
a = splitter(5)
for i in range(10):
    print ( next(a) )

In [34]:
display(query('''select * from nveq.stations where stationid = '101.1.0'  '''))

,sid,stationname,stationid,latitude,longitude,geom
0,7,Engsetvatn,101.1.0,62.53139,6.61767,0101000020E61000009CE1067C7E781A40F2B567960444...
